# Validación cruzada
Una mejor manera de probar tus modelos.

En este tutorial, aprenderá a utilizar la validación cruzada para obtener mejores medidas del rendimiento del modelo.

### Introducción
El aprendizaje automático es un proceso iterativo.

Tendrá que elegir qué variables predictivas utilizar, qué tipos de modelos utilizar, qué argumentos proporcionar a esos modelos, etc. Hasta ahora, ha tomado estas decisiones basándose en datos midiendo la calidad del modelo con una validación ( o reserva) configurado.

Pero este enfoque tiene algunos inconvenientes. Para ver esto, imagina que tienes un conjunto de datos con 5000 filas. Normalmente mantendrá aproximadamente el 20% de los datos como conjunto de datos de validación, o 1000 filas. Pero esto deja cierta posibilidad aleatoria a la hora de determinar las puntuaciones del modelo. Es decir, un modelo podría funcionar bien en un conjunto de 1000 filas, incluso si sería inexacto en 1000 filas diferentes.

En un caso extremo, podría imaginarse tener solo 1 fila de datos en el conjunto de validación. Si comparas modelos alternativos, cuál hace las mejores predicciones sobre un único punto de datos será principalmente una cuestión de suerte.

En general, cuanto mayor sea el conjunto de validación, menos aleatoriedad (también conocida como "ruido") habrá en nuestra medida de la calidad del modelo y más confiable será. Desafortunadamente, solo podemos obtener un conjunto de validación grande eliminando filas de nuestros datos de entrenamiento, ¡y conjuntos de datos de entrenamiento más pequeños significan peores modelos!

### ¿Qué es la validación cruzada?
En la validación cruzada, ejecutamos nuestro proceso de modelado en diferentes subconjuntos de datos para obtener múltiples medidas de la calidad del modelo.

Por ejemplo, podríamos comenzar dividiendo los datos en 5 partes, cada una de las cuales representa el 20% del conjunto de datos completo. En este caso decimos que hemos dividido los datos en 5 "pliegues".

Luego, realizamos un experimento para cada pliegue:

. En el Experimento 1, utilizamos el primer pliegue como conjunto de validación (o reserva) y todo lo demás como datos de entrenamiento. Esto nos da una medida de la calidad del modelo basada en un conjunto de reservas del 20%.

. En el Experimento 2, guardamos datos del segundo pliegue (y usamos todo excepto el segundo pliegue para entrenar el modelo). Luego, el conjunto de reservas se utiliza para obtener una segunda estimación de la calidad del modelo.

. Repetimos este proceso, usando cada pliegue una vez como conjunto de reserva. En conjunto, el 100 % de los datos se utiliza como reserva en algún momento, y terminamos con una medida de la calidad del modelo que se basa en todas las filas del conjunto de datos (incluso si no usamos todas las filas simultáneamente) .

### ¿Cuándo debería utilizar la validación cruzada?
La validación cruzada brinda una medida más precisa de la calidad del modelo, lo cual es especialmente importante si se toman muchas decisiones de modelado. Sin embargo, puede tardar más en ejecutarse porque estima varios modelos (uno para cada pliegue).

Entonces, dadas estas compensaciones, ¿cuándo debería utilizar cada enfoque?

. Para conjuntos de datos pequeños, donde la carga computacional adicional no es gran cosa, debe ejecutar una validación cruzada.

. Para conjuntos de datos más grandes, un único conjunto de validación es suficiente. Su código se ejecutará más rápido y es posible que tenga suficientes datos como para que no sea necesario reutilizar algunos de ellos para reservarlos.

No existe un umbral sencillo para determinar qué constituye un conjunto de datos grande o pequeño. Pero si su modelo tarda un par de minutos o menos en ejecutarse, probablemente valga la pena cambiar a la validación cruzada.

Alternativamente, puede ejecutar una validación cruzada y ver si las puntuaciones de cada experimento parecen cercanas. Si cada experimento arroja los mismos resultados, probablemente un único conjunto de validación sea suficiente.

### Ejemplo
Trabajaremos con los mismos datos que en el tutorial anterior. Cargamos los datos de entrada en X y los datos de salida en y.

In [2]:
import pandas as pd

# Read the data
data = pd.read_csv('melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

Luego, definimos una canalización que utiliza un imputador para completar los valores faltantes y un modelo de bosque aleatorio para hacer predicciones.

Si bien es posible realizar una validación cruzada sin canalizaciones, ¡es bastante difícil! El uso de una canalización hará que el código sea notablemente sencillo.

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

Obtenemos las puntuaciones de validación cruzada con la función cross_val_score() de scikit-learn. Establecemos el número de pliegues con el parámetro cv.

In [5]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


El parámetro de puntuación elige una medida de la calidad del modelo para informar: en este caso, elegimos el error absoluto medio negativo (MAE). Los documentos de scikit-learn muestran una lista de opciones.

Es un poco sorprendente que especifiquemos MAE negativo. Scikit-learn tiene una convención en la que todas las métricas están definidas, por lo que un número alto es mejor. El uso de negativos aquí les permite ser consistentes con esa convención, aunque el MAE negativo es casi inaudito en otros lugares.

Normalmente queremos una medida única de la calidad del modelo para comparar modelos alternativos. Entonces tomamos el promedio de los experimentos.

In [6]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
277707.3795913405


### Conclusión
El uso de la validación cruzada produce una medida mucho mejor de la calidad del modelo, con el beneficio adicional de limpiar nuestro código: tenga en cuenta que ya no necesitamos realizar un seguimiento de los conjuntos de capacitación y validación por separado. Entonces, especialmente para conjuntos de datos pequeños, ¡es una buena mejora!

# EJERCICIOS

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
train_data = pd.read_csv('../input/train.csv', index_col='Id')
test_data = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice              
train_data.drop(['SalePrice'], axis=1, inplace=True)

# Select numeric columns only
numeric_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]
X = train_data[numeric_cols].copy()
X_test = test_data[numeric_cols].copy()

Utilice la siguiente celda de código para imprimir las primeras filas de los datos.

In [ ]:
X.head()

Hasta ahora, ha aprendido cómo construir canalizaciones con scikit-learn. Por ejemplo, la canalización siguiente utilizará SimpleImputer() para reemplazar los valores faltantes en los datos, antes de usar RandomForestRegressor() para entrenar un modelo de bosque aleatorio para hacer predicciones. Establecemos la cantidad de árboles en el modelo de bosque aleatorio con el parámetro n_estimators, y establecer random_state garantiza la reproducibilidad.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

También aprendió a utilizar canalizaciones en validación cruzada. El siguiente código utiliza la función cross_val_score() para obtener el error absoluto medio (MAE), promediado en cinco pliegues diferentes. Recuerde que configuramos el número de pliegues con el parámetro cv.

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("Average MAE score:", scores.mean())

    Average MAE score: 18276.410356164386

### Paso 1: escribe una función útil
En este ejercicio, utilizará la validación cruzada para seleccionar parámetros para un modelo de aprendizaje automático.

Comience escribiendo una función get_score() que informe el MAE promedio (más de tres pliegues de validación cruzada) de una canalización de aprendizaje automático que utiliza:

los datos en X e y para crear pliegues,
SimpleImputer() (con todos los parámetros dejados por defecto) para reemplazar los valores faltantes, y
RandomForestRegressor() (con random_state=0) para ajustarse a un modelo de bosque aleatorio.
El parámetro n_estimators proporcionado a get_score() se utiliza al configurar el número de árboles en el modelo de bosque aleatorio.

In [ ]:
def get_score(n_estimators):
    my_pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', RandomForestRegressor(n_estimators, random_state=0))
    ])
    scores = -1 * cross_val_score(my_pipeline, X, y,
                                  cv=3,
                                  scoring='neg_mean_absolute_error')
    return scores.mean()

### Paso 2: Pruebe diferentes valores de parámetros
Ahora, utilizará la función que definió en el Paso 1 para evaluar el rendimiento del modelo correspondiente a ocho valores diferentes para la cantidad de árboles en el bosque aleatorio: 50, 100, 150, ..., 300, 350, 400.

Almacene sus resultados en los resultados de un diccionario de Python, donde resultados[i] es el MAE promedio devuelto por get_score(i).

In [ ]:
results = {}
for i in range(1,9):
    results[50*i] = get_score(50*i)

Utilice la siguiente celda para visualizar los resultados del Paso 2. Ejecute el código sin cambios.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(list(results.keys()), list(results.values()))
plt.show()

### Paso 3: encuentre el mejor valor de parámetro
Dados los resultados, ¿qué valor de n_estimators parece mejor para el modelo de bosque aleatorio? Utilice su respuesta para establecer el valor de n_estimators_best.

In [ ]:
n_estimators_best = min(results, key=results.get)

En este ejercicio, ha explorado un método para elegir los parámetros adecuados en un modelo de aprendizaje automático.

Si desea obtener más información sobre la optimización de hiperparámetros, le recomendamos que comience con la búsqueda en cuadrícula, que es un método sencillo para determinar la mejor combinación de parámetros para un modelo de aprendizaje automático. Afortunadamente, scikit-learn también contiene una función incorporada GridSearchCV() que puede hacer que su código de búsqueda en cuadrícula sea muy eficiente.